In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
from model import model 
from trainer import trainer

from data_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = get_CIFAR10_data()
for k, v in list(data.items()):
  print(('%s: ' % k, v.shape))

('X_train: ', (49000, 3, 32, 32))
('y_train: ', (49000,))
('X_val: ', (1000, 3, 32, 32))
('y_val: ', (1000,))
('X_test: ', (1000, 3, 32, 32))
('y_test: ', (1000,))


In [13]:
layer_list = ["affine","relu","affine","relu","affine","relu","affine","relu","affine"]
layer_dim = [(3*32*32,100),None,(100,100),None,(100,100),None,(100,100),None,(100,10)]
loss_type = "softmax"



# hyper-param optimizer
low_bound = 1e-4
high_bound = 5e-2
check_values = 10
epochs = 1

max_train = 0
best_lr = 0.0
best_ws = 0.0

for i in range (check_values):
    lr = round(np.random.uniform(low_bound,high_bound),5)
    ws = round(np.random.uniform(low_bound,high_bound),5)
    print(lr, ws)
    my_model = model(layer_list, layer_dim, loss_type, learning_rate=1e-3, update_rule="adam", weight_scale=ws, reg=1e-3)
    my_trainer = trainer(my_model, data, num_epochs=epochs, batch_size=1000)
    my_trainer.train()
    if (my_trainer.train_history[-1] > max_train):
        max_train = my_trainer.train_history[-1]
        best_lr = lr
        best_ws = ws

print("BEST: ", best_lr, best_ws, max_train)


    


0.02963 0.03083
--------- EPOCH: 0 ---------
train acc: 0.096000    validation acc: 0.172000  loss: 2.516770
train acc: 0.314000    validation acc: 0.355000  loss: 2.071633
train acc: 0.370000    validation acc: 0.372000  loss: 1.918441
train acc: 0.407000    validation acc: 0.408000  loss: 1.783918
train acc: 0.443000    validation acc: 0.421000  loss: 1.784086
0.03481 0.01882
--------- EPOCH: 0 ---------
train acc: 0.075000    validation acc: 0.209000  loss: 2.363344
train acc: 0.292000    validation acc: 0.293000  loss: 2.026530
train acc: 0.336000    validation acc: 0.363000  loss: 1.921135
train acc: 0.394000    validation acc: 0.367000  loss: 1.744373
train acc: 0.422000    validation acc: 0.412000  loss: 1.739872
0.0264 0.01649
--------- EPOCH: 0 ---------
train acc: 0.090000    validation acc: 0.164000  loss: 2.348893
train acc: 0.293000    validation acc: 0.301000  loss: 1.984817
train acc: 0.335000    validation acc: 0.362000  loss: 1.862806
train acc: 0.362000    validation 

In [16]:
my_model = model(layer_list, layer_dim, loss_type, learning_rate=1e-3, update_rule="adam", weight_scale=best_ws, reg=1e-4)
my_trainer = trainer(my_model, data, num_epochs=15, batch_size=1000)
my_trainer.train()

--------- EPOCH: 0 ---------
train acc: 0.103000    validation acc: 0.187000  loss: 2.350033
train acc: 0.307000    validation acc: 0.328000  loss: 1.983791
train acc: 0.326000    validation acc: 0.371000  loss: 1.817383
train acc: 0.416000    validation acc: 0.405000  loss: 1.699998
train acc: 0.402000    validation acc: 0.421000  loss: 1.660879
--------- EPOCH: 1 ---------
train acc: 0.461000    validation acc: 0.448000  loss: 1.540730
train acc: 0.427000    validation acc: 0.459000  loss: 1.629682
train acc: 0.467000    validation acc: 0.474000  loss: 1.506288
train acc: 0.463000    validation acc: 0.452000  loss: 1.536243
train acc: 0.485000    validation acc: 0.445000  loss: 1.461543
--------- EPOCH: 2 ---------
train acc: 0.475000    validation acc: 0.470000  loss: 1.496251
train acc: 0.486000    validation acc: 0.486000  loss: 1.486529
train acc: 0.505000    validation acc: 0.475000  loss: 1.425074
train acc: 0.467000    validation acc: 0.493000  loss: 1.509599
train acc: 0.5150

In [23]:
from updater import * 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))



N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
m = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)
v = np.linspace(0.7, 0.5, num=N*D).reshape(N, D)

W = {}
DW = {}
M = {}
V = {}
W["w"] = w
DW["w"] = dw
M["w"] = m
V["w"] = v


my_updater = updater(1e-2,"adam",params=W)
my_updater.first_moment = M
my_updater.sec_moment = V
my_updater.itr = 5


x = my_updater.adam(W, DW)
next_w = x["w"]

expected_next_w = np.asarray([
  [-0.40094747, -0.34836187, -0.29577703, -0.24319299, -0.19060977],
  [-0.1380274,  -0.08544591, -0.03286534,  0.01971428,  0.0722929],
  [ 0.1248705,   0.17744702,  0.23002243,  0.28259667,  0.33516969],
  [ 0.38774145,  0.44031188,  0.49288093,  0.54544852,  0.59801459]])
expected_v = np.asarray([
  [ 0.69966,     0.68908382,  0.67851319,  0.66794809,  0.65738853,],
  [ 0.64683452,  0.63628604,  0.6257431,   0.61520571,  0.60467385,],
  [ 0.59414753,  0.58362676,  0.57311152,  0.56260183,  0.55209767,],
  [ 0.54159906,  0.53110598,  0.52061845,  0.51013645,  0.49966,   ]])
expected_m = np.asarray([
  [ 0.48,        0.49947368,  0.51894737,  0.53842105,  0.55789474],
  [ 0.57736842,  0.59684211,  0.61631579,  0.63578947,  0.65526316],
  [ 0.67473684,  0.69421053,  0.71368421,  0.73315789,  0.75263158],
  [ 0.77210526,  0.79157895,  0.81105263,  0.83052632,  0.85      ]])

print ('next_w error: ', rel_error(expected_next_w, next_w))
print ('v error: ', rel_error(expected_v, my_updater.sec_moment["w"]))
print ('m error: ', rel_error(expected_m, my_updater.first_moment["w"]))

next_w error:  1.09662762797e-07
v error:  4.20831403811e-09
m error:  4.21496319311e-09


In [8]:
from updater import *
N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
cache = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)

W = {}
DW = {}
sq = {}
W["w"] = w
DW["w"] = dw
sq["w"] = cache



my_updater = updater(1e-2,"rmsprop",params=W)
my_updater.grads_sqrd = sq
x = my_updater.rmsprop(W, DW)
next_w = x["w"]


expected_next_w = np.asarray([
  [-0.39223849, -0.34037513, -0.28849239, -0.23659121, -0.18467247],
  [-0.132737,   -0.08078555, -0.02881884,  0.02316247,  0.07515774],
  [ 0.12716641,  0.17918792,  0.23122175,  0.28326742,  0.33532447],
  [ 0.38739248,  0.43947102,  0.49155973,  0.54365823,  0.59576619]])
expected_cache = np.asarray([
  [ 0.5976,      0.6126277,   0.6277108,   0.64284931,  0.65804321],
  [ 0.67329252,  0.68859723,  0.70395734,  0.71937285,  0.73484377],
  [ 0.75037008,  0.7659518,   0.78158892,  0.79728144,  0.81302936],
  [ 0.82883269,  0.84469141,  0.86060554,  0.87657507,  0.8926    ]])

print('next_w error: ', rel_error(expected_next_w, next_w))
print('cache error: ', rel_error(expected_cache,  my_updater.grads_sqrd['w']))

next_w error:  9.78754489423e-08
cache error:  2.64779558072e-09


In [18]:
np.random.seed(231)
N, D, H, C = 3, 5, 50, 7
X = np.random.randn(N, D)
y = np.random.randint(C, size=N)

std = 1e-3

layer_list = ["affine","relu","affine"]
layer_dim = [(D,H),None,(H,C)]
loss_type = "softmax"

my_model = model(layer_list, layer_dim, loss_type, weight_scale=std, reg=0)

print('Testing initialization ... ')
W1_std = abs(my_model.layers[0].params['W'].std() - std)
b1 = my_model.layers[0].params['b']
W2_std = abs(my_model.layers[2].params['W'].std() - std)
b2 = my_model.layers[2].params['b']
assert W1_std < std / 10, 'First layer weights do not seem right'
assert np.all(b1 == 0), 'First layer biases do not seem right'
assert W2_std < std / 10, 'Second layer weights do not seem right'
assert np.all(b2 == 0), 'Second layer biases do not seem right'

print('Testing test-time forward pass ... ')
my_model.layers[0].params['W'] = np.linspace(-0.7, 0.3, num=D*H).reshape(D, H)
my_model.layers[0].params['b'] = np.linspace(-0.1, 0.9, num=H)
my_model.layers[2].params['W'] = np.linspace(-0.3, 0.4, num=H*C).reshape(H, C)
my_model.layers[2].params['b'] = np.linspace(-0.9, 0.1, num=C)
X = np.linspace(-5.5, 4.5, num=N*D).reshape(D, N).T
scores, w = my_model.forward_pass(X)
correct_scores = np.asarray(
  [[11.53165108,  12.2917344,   13.05181771,  13.81190102,  14.57198434, 15.33206765,  16.09215096],
   [12.05769098,  12.74614105,  13.43459113,  14.1230412,   14.81149128, 15.49994135,  16.18839143],
   [12.58373087,  13.20054771,  13.81736455,  14.43418138,  15.05099822, 15.66781506,  16.2846319 ]])
scores_diff = np.abs(scores - correct_scores).sum()
assert scores_diff < 1e-6, 'Problem with test-time forward pass'

print('Testing training loss (no regularization)')
y = np.asarray([0, 5, 1])
grads, loss = my_model.calculate_loss(scores, y, w)
correct_loss = 3.4702243556

assert abs(loss - correct_loss) < 1e-10, 'Problem with training-time loss'

my_model.reg = 1.0
for layer in my_model.layers:
    layer.reg = 1.0

scores, w = my_model.forward_pass(X)
grads, loss = my_model.calculate_loss(scores, y, w)
correct_loss = 26.5948426952
print(abs(loss - correct_loss))
assert abs(loss - correct_loss) < 1e-10, 'Problem with regularization loss'



Testing initialization ... 
Testing test-time forward pass ... 
Testing training loss (no regularization)
3.85824705518e-11
Running numeric gradient check with reg =  0.0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
from layers import * 
# Check the training-time forward pass by checking means and variances
# of features both before and after batch normalization

# Simulate the forward pass for a two-layer network
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print ('Before batch normalization:')
print ('  means: ', a.mean(axis=0))
print ('  stds: ', a.std(axis=0))
bnl = batch_norm_layer((D3,D3), 1e-3, "", "")
bnl.params["gamma"] = np.ones(D3)
bnl.params["beta"] = np.zeros(D3)

# Means should be close to zero and stds close to one
print ('After batch normalization (gamma=1, beta=0)')
a_norm = bnl.step_forward(a)
print ('  mean: ', a_norm.mean(axis=0))
print ('  std: ', a_norm.std(axis=0))

# Now means should be close to beta and stds close to gamma
gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
bnl.params["gamma"] = gamma
bnl.params["beta"] = beta
a_norm = bnl.step_forward(a)
print ('After batch normalization (nontrivial gamma, beta)')
print ('  means: ', a_norm.mean(axis=0))
print ('  stds: ', a_norm.std(axis=0))

Before batch normalization:
  means:  [ 26.55068073 -13.66394309   0.92234417]
  stds:  [ 34.83511196  33.69398718  24.00501513]
After batch normalization (gamma=1, beta=0)
  mean:  [ -5.88626370e-16  -1.38777878e-17  -3.33066907e-18]
  std:  [ 1.          1.          0.99999999]
After batch normalization (nontrivial gamma, beta)
  means:  [ 11.  12.  13.]
  stds:  [ 1.          1.99999999  2.99999997]
